In [1]:
#[Hipótese 1] Data por CNAE

In [2]:
# Maior movimentação, elencar o maior grupo de tipos de empresa.
# 1. Criar faixas de horário de 3 em 3 horas com transações efetuadas nesse horário - por empresa.
# 2. Plotar histograma de faixa de horário de transação. 1 gráfico por CNAE (não empresa).
# 3. Verificar a possibilidade de clusterização utilizando KNN caso gráfico de um CNAE seja muito divergente 
#    (sem faixa de horários muito contrastantes), verificando se cabem clusters de clientes, dado horário de transações.

# Faixas de horário:
# 02 h as 05 h
# 05 h as 08 h
# 08 h as 11 h
# 11 h as 14 h
# 14 h as 17 h
# 17 h as 20 h
# 20 h as 23 h
# 23 h as 02 h

In [3]:
# importar arquivo como csv

import pandas as pd

combined_csv = pd.read_csv("../combined_csv.csv", skiprows = 1, names =["dominio","empresa","3","datahora","5","6","7","8"])

print(combined_csv.shape)

D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(1499999, 8)


In [4]:
# removendo colunas desnecessarias

combined_csv.drop("3", inplace=True, axis=1)
combined_csv.drop("5", inplace=True, axis=1)
combined_csv.drop("6", inplace=True, axis=1)
combined_csv.drop("7", inplace=True, axis=1)
combined_csv.drop("8", inplace=True, axis=1)

In [5]:
# tratamento de data e hora

from datetime import datetime

data_e_hora_parsed = pd.to_datetime(combined_csv.loc[:, "datahora"], format="%Y-%m-%dT%H:%M:%S.000Z",errors="coerce")

combined_csv["datahora"] = data_e_hora_parsed

print(combined_csv.head(4))

                                             dominio  \
0  6139646137356433363237396637323937336161383832...   
1  6139646137356433363237396637323937336161383832...   
2  6139646137356433363237396637323937336161383832...   
3  6139646137356433363237396637323937336161383832...   

                        empresa            datahora  
0  21404ebaa0106ca20ae283c89271 2018-11-19 16:40:45  
1  510766392a95afd41c6bd526381a 2018-12-07 09:56:37  
2  21404ebaa0106ca20ae283c89271 2018-10-18 08:29:47  
3  8ef4d40b852168ea8e3bbc4c89a0 2018-10-15 09:18:55  


In [6]:
# limpeza dos campos NaT

df_datas_corretas = combined_csv[combined_csv["datahora"].notnull()] 
df_datas_incorretas = combined_csv[combined_csv["datahora"].isnull()]

print(df_datas_corretas.shape)
print(df_datas_corretas.head(4))

(1499980, 3)
                                             dominio  \
0  6139646137356433363237396637323937336161383832...   
1  6139646137356433363237396637323937336161383832...   
2  6139646137356433363237396637323937336161383832...   
3  6139646137356433363237396637323937336161383832...   

                        empresa            datahora  
0  21404ebaa0106ca20ae283c89271 2018-11-19 16:40:45  
1  510766392a95afd41c6bd526381a 2018-12-07 09:56:37  
2  21404ebaa0106ca20ae283c89271 2018-10-18 08:29:47  
3  8ef4d40b852168ea8e3bbc4c89a0 2018-10-15 09:18:55  


In [7]:
# transformando em horário

df_horarios_transacoes = df_datas_corretas["datahora"].dt.hour

df_datas_corretas["hora"] = df_horarios_transacoes

df_datas_corretas.drop("datahora", inplace=True, axis=1)

print(df_datas_corretas.shape)
print(df_datas_corretas.head(4))

(1499980, 3)
                                             dominio  \
0  6139646137356433363237396637323937336161383832...   
1  6139646137356433363237396637323937336161383832...   
2  6139646137356433363237396637323937336161383832...   
3  6139646137356433363237396637323937336161383832...   

                        empresa  hora  
0  21404ebaa0106ca20ae283c89271    16  
1  510766392a95afd41c6bd526381a     9  
2  21404ebaa0106ca20ae283c89271     8  
3  8ef4d40b852168ea8e3bbc4c89a0     9  


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
D:\Anaconda\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [30]:
# Agrupando as empresas e criando um dataframe de horário por cada transação

empresas = df_datas_corretas.empresa.unique()

COLUNAS = [
    '0h',
    '1h',
    '2h',
    '3h',
    '4h',
    '5h',
    '6h',
    '7h',
    '8h',
    '9h',
    '10h',
    '11h',
    '12h',
    '13h',
    '14h',
    '15h',
    '16h',
    '17h',
    '18h',
    '19h',
    '20h',
    '21h',
    '22h',
    '23h',
    'Total',
    'Erros'
]

frequencia_horario_empresas = pd.DataFrame(columns=COLUNAS,index=empresas)

for col in frequencia_horario_empresas.columns:
    frequencia_horario_empresas[col].values[:] = 0
    
print(frequencia_horario_empresas.head(3))

                             0h 1h 2h 3h 4h 5h 6h 7h 8h 9h  ... 16h 17h 18h  \
21404ebaa0106ca20ae283c89271  0  0  0  0  0  0  0  0  0  0  ...   0   0   0   
510766392a95afd41c6bd526381a  0  0  0  0  0  0  0  0  0  0  ...   0   0   0   
8ef4d40b852168ea8e3bbc4c89a0  0  0  0  0  0  0  0  0  0  0  ...   0   0   0   

                             19h 20h 21h 22h 23h Total Erros  
21404ebaa0106ca20ae283c89271   0   0   0   0   0     0     0  
510766392a95afd41c6bd526381a   0   0   0   0   0     0     0  
8ef4d40b852168ea8e3bbc4c89a0   0   0   0   0   0     0     0  

[3 rows x 26 columns]


In [31]:
# populando o dataframe de frequencias de horario por empresa

for index, row in df_datas_corretas.iterrows():
    if index % 100000 == 0:
        print(index)
    if row["hora"] == 0:
        frequencia_horario_empresas.loc[row["empresa"], "0h"] += 1
    elif row["hora"] == 1:
        frequencia_horario_empresas.loc[row["empresa"], "1h"] += 1
    elif row["hora"] == 2:
        frequencia_horario_empresas.loc[row["empresa"], "2h"] += 1
    elif row["hora"] == 3:
        frequencia_horario_empresas.loc[row["empresa"], "3h"] += 1
    elif row["hora"] == 4:
        frequencia_horario_empresas.loc[row["empresa"], "4h"] += 1
    elif row["hora"] == 5:
        frequencia_horario_empresas.loc[row["empresa"], "5h"] += 1
    elif row["hora"] == 6:
        frequencia_horario_empresas.loc[row["empresa"], "6h"] += 1
    elif row["hora"] == 7:
        frequencia_horario_empresas.loc[row["empresa"], "7h"] += 1
    elif row["hora"] == 8:
        frequencia_horario_empresas.loc[row["empresa"], "8h"] += 1
    elif row["hora"] == 9:
        frequencia_horario_empresas.loc[row["empresa"], "9h"] += 1
    elif row["hora"] == 10:
        frequencia_horario_empresas.loc[row["empresa"], "10h"] += 1
    elif row["hora"] == 11:
        frequencia_horario_empresas.loc[row["empresa"], "11h"] += 1
    elif row["hora"] == 12:
        frequencia_horario_empresas.loc[row["empresa"], "12h"] += 1
    elif row["hora"] == 13:
        frequencia_horario_empresas.loc[row["empresa"], "13h"] += 1
    elif row["hora"] == 14:
        frequencia_horario_empresas.loc[row["empresa"], "14h"] += 1
    elif row["hora"] == 15:
        frequencia_horario_empresas.loc[row["empresa"], "15h"] += 1
    elif row["hora"] == 16:
        frequencia_horario_empresas.loc[row["empresa"], "16h"] += 1
    elif row["hora"] == 17:
        frequencia_horario_empresas.loc[row["empresa"], "17h"] += 1
    elif row["hora"] == 18:
        frequencia_horario_empresas.loc[row["empresa"], "18h"] += 1
    elif row["hora"] == 19:
        frequencia_horario_empresas.loc[row["empresa"], "19h"] += 1
    elif row["hora"] == 20:
        frequencia_horario_empresas.loc[row["empresa"], "20h"] += 1
    elif row["hora"] == 21:
        frequencia_horario_empresas.loc[row["empresa"], "21h"] += 1
    elif row["hora"] == 22:
        frequencia_horario_empresas.loc[row["empresa"], "22h"] += 1
    elif row["hora"] == 23:
        frequencia_horario_empresas.loc[row["empresa"], "23h"] += 1
    else:
        frequencia_horario_empresas.loc[row["empresa"], "Erros"] += 1
        
    frequencia_horario_empresas.loc[row["empresa"], "Total"] += 1
        

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000


In [32]:
# Checando se resultado bate com valor total

print(frequencia_horario_empresas.head(2))

total = 0
erros = 0

for index, row in frequencia_horario_empresas.iterrows():
    total += row["Total"]
    erros += row["Erros"]
    
print(total)
print(erros)

                               0h  1h  2h  3h  4h  5h   6h   7h   8h   9h  \
21404ebaa0106ca20ae283c89271  190  34  44  32  38  95  242  252  219  379   
510766392a95afd41c6bd526381a   29  26   8   8   9   7  141  237  259  272   

                              ...  16h  17h   18h  19h  20h  21h  22h  23h  \
21404ebaa0106ca20ae283c89271  ...  606  805  1018  936  784  507  464  574   
510766392a95afd41c6bd526381a  ...  268  285   320  288  301  281  187  124   

                              Total  Erros  
21404ebaa0106ca20ae283c89271  11098      0  
510766392a95afd41c6bd526381a   4761      0  

[2 rows x 26 columns]
1499980
0


In [33]:
# Exportando arquivo de frequencias

frequencia_horario_empresas.drop("Erros", inplace=True, axis=1)

export_csv = frequencia_horario_empresas.to_csv (r'frequencia_transacoes_por_empresa.csv', index_label = "empresa")

print(frequencia_horario_empresas.shape)

(1548, 25)


In [35]:
# Fazendo o cálculo de % de transações por faixa de horário

porcentagem_frequencia_empresas = frequencia_horario_empresas.copy()

for index, row in frequencia_horario_empresas.iterrows():
    porcentagem_frequencia_empresas.loc[index,["0h"]]  = row["0h"]  / row["Total"]
    porcentagem_frequencia_empresas.loc[index,["1h"]]  = row["1h"]  / row["Total"]
    porcentagem_frequencia_empresas.loc[index,["2h"]]  = row["2h"]  / row["Total"]
    porcentagem_frequencia_empresas.loc[index,["3h"]]  = row["3h"]  / row["Total"]
    porcentagem_frequencia_empresas.loc[index,["4h"]]  = row["4h"]  / row["Total"]
    porcentagem_frequencia_empresas.loc[index,["5h"]]  = row["5h"]  / row["Total"]
    porcentagem_frequencia_empresas.loc[index,["6h"]]  = row["6h"]  / row["Total"]
    porcentagem_frequencia_empresas.loc[index,["7h"]]  = row["7h"]  / row["Total"]
    porcentagem_frequencia_empresas.loc[index,["8h"]]  = row["8h"]  / row["Total"]
    porcentagem_frequencia_empresas.loc[index,["9h"]]  = row["9h"]  / row["Total"]
    porcentagem_frequencia_empresas.loc[index,["10h"]] = row["10h"] / row["Total"]
    porcentagem_frequencia_empresas.loc[index,["11h"]] = row["11h"] / row["Total"]
    porcentagem_frequencia_empresas.loc[index,["12h"]] = row["12h"] / row["Total"]
    porcentagem_frequencia_empresas.loc[index,["13h"]] = row["13h"] / row["Total"]
    porcentagem_frequencia_empresas.loc[index,["14h"]] = row["14h"] / row["Total"]
    porcentagem_frequencia_empresas.loc[index,["15h"]] = row["15h"] / row["Total"]
    porcentagem_frequencia_empresas.loc[index,["16h"]] = row["16h"] / row["Total"]
    porcentagem_frequencia_empresas.loc[index,["17h"]] = row["17h"] / row["Total"]
    porcentagem_frequencia_empresas.loc[index,["18h"]] = row["18h"] / row["Total"]
    porcentagem_frequencia_empresas.loc[index,["19h"]] = row["19h"] / row["Total"]
    porcentagem_frequencia_empresas.loc[index,["20h"]] = row["20h"] / row["Total"]
    porcentagem_frequencia_empresas.loc[index,["21h"]] = row["21h"] / row["Total"]
    porcentagem_frequencia_empresas.loc[index,["22h"]] = row["22h"] / row["Total"]
    porcentagem_frequencia_empresas.loc[index,["23h"]] = row["23h"] / row["Total"]

print(frequencia_horario_empresas.head(5))
print(porcentagem_frequencia_empresas.head(5))

                               0h   1h   2h  3h   4h   5h   6h   7h   8h   9h  \
21404ebaa0106ca20ae283c89271  190   34   44  32   38   95  242  252  219  379   
510766392a95afd41c6bd526381a   29   26    8   8    9    7  141  237  259  272   
8ef4d40b852168ea8e3bbc4c89a0  177  166  108  75  106  127  186  262  261  284   
617b9dcd2479b7e85da7a90b56c4   23   16   20  15   18   47   63   92  125  135   
5dc07a1ef6f3839d858729107f26    0    0    0   0    0    1   41  132  110  110   

                              ...  15h  16h  17h   18h  19h  20h  21h  22h  \
21404ebaa0106ca20ae283c89271  ...  603  606  805  1018  936  784  507  464   
510766392a95afd41c6bd526381a  ...  292  268  285   320  288  301  281  187   
8ef4d40b852168ea8e3bbc4c89a0  ...  285  325  316   318  343  315  260  266   
617b9dcd2479b7e85da7a90b56c4  ...  135  122  175   153  152  146  115   70   
5dc07a1ef6f3839d858729107f26  ...  105   93  136   129  122   69   57    9   

                              23h  Total  
2

In [36]:
# Exportando arquivo de porcentagens

export_csv = porcentagem_frequencia_empresas.to_csv (r'porcentagem_transacoes_por_empresa.csv', index_label = "empresa")

print(porcentagem_frequencia_empresas.shape)

(1548, 25)
